# Homework 9
12110418 
庄子鲲
## Problem 1
Using Python and Numpy, write a class named SVMClassifier, which implements the SVM algorithm having slack variables and kernels such as Polynomial,Gaussian, and Sigmoid (using cvxopt package to solve the quadratic programing problem for Lagrange multipliers).

For a svm with slack variables and kernels:
Optimize function:
$$
\max_{a_i\ge 0}\{\sum_{i=1}^{N}\alpha _i-\frac{1}{2}\sum_{i,j=1}^{N}t^{(i)}t^{(j)}\alpha^i \alpha ^j K(\mathbf x^{(i)},\mathbf x^{(j)}) \}\\
$$
where
$$
0\le \alpha^{(i)}\le -\xi^{(i)}\\
\xi^{(i)}\ge0
$$
Linear kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\mathbf x^{(i)^{T}} \mathbf x^{(j)}
$$
$d^{\text {th}}$ degree of Polynomial kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=(\mathbf x^{(i)}\mathbf x^{(j)}+1)^d
$$
Gaussian kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\exp(-\frac{||\mathbf x^{(i)}-\mathbf x^{(j)}||^2}{2\sigma ^2})
$$
Sigmoid kernel:
$$
K(\mathbf x^{(i)},\mathbf x^{(j)})=\tanh (\beta (\mathbf x^{(i)^{T}}\mathbf x^{(j)}))
$$
Output:
$$
y=\text {sign}[b+\mathbf x\cdot (\sum_{i=1}^{(N)}\alpha_it^{(i)}\mathbf x^{(i)})]
$$


In [1]:
import numpy as np
from cvxopt import matrix, solvers

class SVMClassifier:
    def __init__(self, kernel='linear', degree=3, alpha_threshold=1e-5,gamma=None, coef0=0.0, C=1.0):
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.C = C
        self.alpha_threshold = alpha_threshold
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.bias = None

    def linear_kernel(self, x1, x2):
        return np.dot(x1, x2.T)
    
    def poly_kernel(self, x1, x2):
        return (np.dot(x1, x2.T) + self.coef0)**self.degree
    
    def rbf_kernel(self, x1, x2):
        return np.exp(-((x1[:, np.newaxis] - x2)**2).sum(axis=2)/(2 * self.gamma**2))
    
    def sigmoid_kernel(self, x1, x2):
        return np.tanh(self.gamma * np.dot(x1, x2.T) + self.coef0)
    
    def fit(self, X, y):
        if self.kernel == 'linear':
            self.K = self.linear_kernel(X, X)
        elif self.kernel == 'poly':
            self.K = self.poly_kernel(X, X)
        elif self.kernel == 'rbf':
            if self.gamma is None:
                self.gamma = 1.0 / X.shape[1]  # Default gamma
            self.K = self.rbf_kernel(X, X)
        elif self.kernel == 'sigmoid':
            self.K = self.sigmoid_kernel(X, X)

        n_samples, n_features = X.shape
        P = matrix(np.outer(y, y) * self.K, tc='d')
        q = matrix(-np.ones(n_samples), tc='d')
        G = matrix(np.vstack((np.eye(n_samples), -np.eye(n_samples))), tc='d')
        h = matrix(np.hstack((self.C * np.ones(n_samples), np.zeros(n_samples))), tc='d')
        A = matrix(y.reshape(1, -1), tc='d')
        b = matrix(0.0, tc='d')

        # Solve the quadratic programming problem
        solution = solvers.qp(P, q, G, h, A, b)

        # Extract Lagrange multipliers from the solution
        self.alpha = np.array(solution['x']).flatten()
        # Support vectors have non-zero Lagrange multipliers
        sv_indices = np.where(self.alpha > self.alpha_threshold)[0]
        self.support_vectors = X[sv_indices]
        self.support_vector_labels = y[sv_indices]
        print('number of support vector: ',self.support_vectors.shape[0])
        self.alpha = self.alpha.reshape(-1,1)
        self.alpha=self.alpha[sv_indices]
        # Compute the bias term
        if self.kernel == 'linear':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.linear_kernel(self.support_vectors,self.support_vectors), (self.alpha * self.support_vector_labels)))     
        elif self.kernel == 'poly':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.poly_kernel(self.support_vectors,self.support_vectors), (self.alpha * self.support_vector_labels)))
        elif self.kernel=='rbf':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.rbf_kernel(self.support_vectors,self.support_vectors),(self.alpha * self.support_vector_labels)))
        elif self.kernel=='sigmoid':
            self.bias = np.mean(self.support_vector_labels-np.dot(self.sigmoid_kernel(self.support_vectors,self.support_vectors),(self.alpha * self.support_vector_labels)))
        
        
    def predict(self, X):
        if self.kernel == 'linear':
            decision_function = np.dot(self.linear_kernel(X,self.support_vectors), (self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'poly':
            decision_function = np.dot(self.poly_kernel(X,self.support_vectors), (self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'rbf':
            decision_function = np.dot(self.rbf_kernel(X,self.support_vectors),(self.alpha * self.support_vector_labels)) + self.bias
        elif self.kernel == 'sigmoid':
            decision_function = np.dot(self.sigmoid_kernel(X,self.support_vectors),(self.alpha * self.support_vector_labels)) + self.bias

        # Predict using the sign of the decision function
        return np.sign(decision_function)


## Problem 2
Consider the dataset of letter recognition (named letter-recognition.data).The dataset has 20,000 samples, for which the first column indicates theclass(A~Z,totally 26 classes), and the rest columns indicate 16 features as described in the following table. For this dataset, use SVM to do a binary classification for letter‘C’or non-‘C’class.

1. Linear Kernel:

In [2]:
svm_linear = SVMClassifier(kernel='linear', gamma=0.1, C=1.0)
# Load data from the file
data = np.loadtxt("letter-recognition.data", dtype=str, delimiter=',')
# Extract labels and features
y = np.array([1 if label == "C" else -1 for label in data[:6000, 0]])
y=y.reshape(-1,1)
X = data[:6000, 1:].astype(int)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape[0],'training data is used to train')
svm_linear.fit(X_train, y_train)
predictions = svm_linear.predict(X_test)
print(X_test.shape[0],'testing data to be test')
print('accuracy:', accuracy_score(y_test, predictions))

4800 training data is used to train
     pcost       dcost       gap    pres   dres
 0: -5.7559e+02 -1.0103e+04  6e+04  3e+00  2e-11
 1: -3.7733e+02 -5.4811e+03  9e+03  3e-01  1e-11
 2: -3.3897e+02 -2.5982e+03  4e+03  1e-01  1e-11
 3: -3.0722e+02 -2.1627e+03  3e+03  8e-02  1e-11
 4: -2.7483e+02 -1.3717e+03  2e+03  4e-02  9e-12
 5: -2.6111e+02 -9.1949e+02  1e+03  2e-02  8e-12
 6: -2.5598e+02 -6.7103e+02  6e+02  1e-02  8e-12
 7: -2.6106e+02 -4.9233e+02  3e+02  5e-03  8e-12
 8: -2.6418e+02 -4.4184e+02  2e+02  3e-03  8e-12
 9: -2.6664e+02 -4.0698e+02  2e+02  2e-03  8e-12
10: -2.6859e+02 -3.8642e+02  1e+02  1e-03  8e-12
11: -2.7228e+02 -3.6607e+02  1e+02  1e-03  9e-12
12: -2.7326e+02 -3.5136e+02  8e+01  5e-04  9e-12
13: -2.7617e+02 -3.3622e+02  6e+01  2e-04  1e-11
14: -2.8080e+02 -3.2461e+02  5e+01  1e-04  1e-11
15: -2.8231e+02 -3.2101e+02  4e+01  8e-05  1e-11
16: -2.8468e+02 -3.1531e+02  3e+01  4e-05  1e-11
17: -2.8627e+02 -3.1193e+02  3e+01  3e-05  1e-11
18: -2.8781e+02 -3.0832e+02  2e+01

2. Polynomial Kernel:

In [3]:
svm_poly = SVMClassifier(kernel='poly', alpha_threshold=1e-10,degree=3, gamma=0.1, C=1.0)
svm_poly.fit(X_train, y_train)
predictions = svm_poly.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -1.5936e+02 -9.3055e+03  5e+04  3e+00  5e-06
 1: -1.0231e+02 -5.0491e+03  1e+04  4e-01  5e-06
 2: -6.0150e+01 -2.1735e+03  4e+03  1e-01  3e-06
 3: -3.5221e+01 -1.3557e+03  2e+03  8e-02  2e-06
 4: -2.2088e+01 -9.2339e+02  2e+03  5e-02  1e-06
 5: -1.1831e+01 -5.3100e+02  9e+02  2e-02  8e-07
 6: -4.5851e+00 -4.1067e+02  7e+02  1e-02  5e-07
 7: -2.1619e-01 -2.4166e+01  4e+01  8e-04  1e-07
 8: -2.1936e-02 -3.2150e+00  5e+00  9e-05  2e-08
 9: -6.3982e-03 -6.7029e-01  1e+00  2e-05  4e-09
10: -3.3485e-03 -2.2923e-01  3e-01  5e-06  1e-09
11: -1.9474e-03 -1.0172e-01  1e-01  2e-06  6e-10
12: -1.1739e-03 -4.8356e-02  7e-02  6e-07  3e-10
13: -7.1427e-04 -2.1186e-02  3e-02  2e-07  2e-10
14: -4.3522e-04 -1.0390e-02  1e-02  1e-07  9e-11
15: -3.0935e-04 -5.0005e-03  6e-03  4e-08  7e-11
16: -2.5519e-04 -2.7808e-03  3e-03  2e-08  6e-11
17: -2.0348e-04 -1.6727e-03  2e-03  5e-09  5e-11
18: -2.9682e-04 -1.0811e-03  9e-04  2e-09  5e-11
19: -2.7905e-04 -9.22

3. Guassian Kernel:

In [4]:
svm_rbf = SVMClassifier(kernel='rbf',gamma=0.1, C=1.0)
svm_rbf.fit(X_train, y_train)
predictions = svm_rbf.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -3.3246e+02 -6.8029e+03  3e+04  2e+00  2e-16
 1: -2.9590e+02 -2.8883e+03  3e+03  7e-02  4e-16
 2: -2.8817e+02 -4.3812e+02  2e+02  5e-04  5e-16
 3: -2.8857e+02 -2.9053e+02  2e+00  6e-06  1e-16
 4: -2.8871e+02 -2.8877e+02  5e-02  8e-08  6e-17
 5: -2.8872e+02 -2.8872e+02  2e-03  8e-10  2e-17
 6: -2.8872e+02 -2.8872e+02  2e-05  8e-12  7e-17
Optimal solution found.
number of support vector:  4800
accuracy: 0.9666666666666667


4. Sigmoid Kernel:

In [5]:
svm_sigmoid = SVMClassifier(kernel='sigmoid',gamma=0.1, C=1.0)
svm_sigmoid.fit(X_train, y_train)
predictions = svm_sigmoid.predict(X_test)
print('accuracy:', accuracy_score(y_test, predictions))

     pcost       dcost       gap    pres   dres
 0: -7.5199e+02 -9.9600e+03  5e+04  3e+00  3e-13
 1: -4.2338e+02 -5.1953e+03  6e+03  1e-01  4e-13
 2: -3.9141e+02 -6.9147e+02  3e+02  3e-04  3e-14
 3: -3.9189e+02 -3.9502e+02  3e+00  3e-06  1e-14
 4: -3.9200e+02 -3.9203e+02  3e-02  3e-08  2e-14
 5: -3.9200e+02 -3.9200e+02  3e-04  3e-10  2e-14
Optimal solution found.
number of support vector:  4800
accuracy: 0.9658333333333333
